In [2]:
!pip show jupyterlab

Name: jupyterlab
Version: 1.0.2
Summary: The JupyterLab notebook server extension.
Home-page: http://jupyter.org
Author: Jupyter Development Team
Author-email: jupyter@googlegroups.com
License: BSD
Location: /usr/local/lib/python3.6/dist-packages
Requires: jupyterlab-server, notebook, tornado
Required-by: 


# KENT
## 대화 에이전트

#### kent.agent.Agent(domain_finder, intent_finder)
주어진 도메인 및 의도 분류기를 바탕으로 하는 대화 에이전트를 생성합니다. 대화 에이전트가 입력 발화에 대하여 응답을 생성하기 위해선, 도메인 및 의도 분류 결과에 따라 특정 행동을 할 수 있도록 규칙을 추가해야 합니다. 

##### listen(utterance)
입력된 발화를 대화 에이전트가 분석한 뒤, 정의된 규칙에 따라 수행해야 할 행동을 반환합니다. `utterance`에는 반드시 `kent.utterance.Utterance` 객체를 전달해야 합니다.

```python
>>> utterance = Utterance('가정용 전기 요금에만 누진제를 적용하는 이유가 무엇인가요?')
>>> agent.listen(utterance)
```

##### on(semantic)
대화 에이전트가 입력 발화의 도메인 및 의도 분류 결과에 따라 특정 행동을 할 수 있도록 규칙을 추가하는 Decorator입니다. `kent.understanding.Domain(name, confidence)` 또는 `kent.understanding.Intent(name, confidence)` 객체를 `semantic`으로 전달 할 수 있습니다. 해당 Decorator로 추가된 행동은 *도메인 또는 의도가 `name`일 신뢰도가 `confidence` 이상일 때* 발생합니다. 해당 Decorator를 중첩하여 사용할 경우, AND 조건을 표현할 수 있습니다.

```python
>>> @agent.on(Domain(name='기타/TV/수신료', confidence=0.5))
    @agent.on(Intent(name='문의하다/HOW > 면제/장애인', confidence=0.5))
    def _(utterance):
         return TextAction('장애인의 TV면제 신청방법은 장애인(보건복지부에 등록된 시각 또는 청각 장애인) 증명서나 복지카드를 복사하시고 그 여백에 전기요금청구서상 고객번호와 전화번호를 기재하시어 관할 한전으로 우편이나 FAX로 보내주시면 됩니다.')
```

#### 예시

In [10]:
from kent.agent import Agent
from kent.understanding import DomainFinder, IntentFinder

agent = Agent(domain_finder=DomainFinder('kepco/domain.pickle'), intent_finder=IntentFinder('kepco/intent.pickle'))

In [11]:
import functools
from kent.understanding import Domain as _Domain, Intent as _Intent
from kent.actions import TextAction

Domain = functools.partial(_Domain, confidence=0.5)
Intent = functools.partial(_Intent, confidence=0.5)

@agent.on(Domain('전기/요금'))
@agent.on(Intent('문의하다/HOW > 계산/기본료'))
def _(utterance):
    return TextAction('전기요금은 기본요금과 전력량요금으로 구성되어 있습니다. 이중 기본요금은 발전소, 변전소 등 전력공급설비 투자에 따른 감가상각비, 수선유지비 등 고정비를 충당하기 위해 청구되는 요금입니다. 기본요금은 전기사용 용도(주택용, 일반용, 산업용, 교육용, 농사용 등)에 따라 단가가 다릅니다. 주택용전력은 사용량에 따라 기본요금은 3단계로 구분 적용하고 있으며, 주택용전력을 제외한 계약종별의 기본요금은 계약전력을 기준으로 하므로 계약전력은 요금계산의 기준이 되는 요금적용전력이 됩니다. 다만, 최대수요전력을 계량할 수 있는 전력량계를 설치한 고압 고객은 검침당월을 포함한 직전 12개월 중 12월분, 1월분, 2월분, 7월분, 8월분, 9월분 및 검침당월분의 최대수요전력 중 가장 큰 최대수요전력을 요금적용전력으로 하여 기본요금을 산정합니다. - 기본요금(원) = 요금적용전력(kW) × 전기요금표상 기본요금 단가(원/kW) (다만, 주택용 전력은 월간 전력사용량 구간별 단가 적용) <기본요금 산정기준> 요금 구분 구분 요금적용전력 비고 저압 계약전력 사용량이 없는 달에는 기본요금 50% 감액 고압이상 DM부설 검침 당월을 포함한 직전 12개월중 12월분,1월분,2월분,7월분, 8월분, 9월분 및 검침당월분의 최대수요전력 중 가장 큰 최대수요전력 가장 큰 최대수요전력이 계약전력의 30%미만인 경우에는 계약전력의 30%를 요금적용전력으로 함 DM 미부설 계약전력 사용량이 없는 월에는 기본요금의 50%를 감액 ※ 기본요금 50% 감액 제외 대상 : 휴지고객, 주택용 고객, 가로등(갑) 등의 정액제고객, 심야전력(갑) 고객, 심야전력(을)Ⅰ 고객※ 자세한 계약종별 전기요금은 "사이버지점>조회·납부>전기요금표"를 참고하시기 바랍니다.')


@agent.on(Domain('전기/요금'))
@agent.on(Intent('문의하다/HOW > 계산/전기료'))
def _(utterance):
    return TextAction('전기요금은 기본요금과 전력량요금으로 구성되며, 기본요금과 전력량요금의 합계에 전력산업기반기금과 부가가치세가 포함되어 청구금액이 결정됩니다. 기본요금 및 전력량요금 단가는 전기공급방식(고압, 저압), 계약종별(주택용, 일반용, 산업용, 교육용, 농사용 등)에 따라 다릅니다. 주택용전력은 사용량에 따라 기본요금은 3단계, 전력량요금은 3단계로 구분하여 누진율을 적용합니다. 주택용전력을 제외한 모든 계약종별의 기본요금 적용은 계약전력을 기준으로 하므로 계약전력은 요금계산의 기준이 되는 요금적용전력이 됩니다. 다만, 최대수요전력계를 설치한 고객에 대하여는 검침 당월을 포함한 직전 12개월중 12월분, 1월분, 2월분, 7월분, 8월분, 9월분 및 당월분의 최대수요전력 중 가장 큰 최대수요전력을 요금적용전력으로 하여 기본요금을 산정합니다. "사이버지점>전기요금계산기"을 클릭하시면 사용량에 따른 전기요금을 계산해 보실 수 있습니다. 또 "사이버지점>조회·납부>전기요금표"에서는 계약종별별 전기요금표를 보실 수 있습니다.◆ 주택용전력 저압(주거용) 전기요금 계산 예 : 225kW 사용시① 기본요금 : 200kWh초과로 1,600원 ② 전력량요금 : 23,357원 - 처음 200kWh×93.3원=18,660원 - 다음 25kWh×187.9원=4,697.5원 (원미만 절사)③ 요금합계 (기본요금+전력량요금) : 1,600원+23,357원=24,957원 ④ 부가가치세 (요금합계*10%) : 24,957원×0.1=2,496원 (원미만 4사5입)⑤ 전력산업기반기금 (요금합계×3.7%) : 24,957원×0.037=920원 (10원미만절사)⑥ 청구금액：24,957원+2,496원+920원=28,370원 (국고금단수법에 의해 10원미만 절사)')


@agent.on(Domain('전기/요금'))
@agent.on(Intent('문의하다/WHY > 누진제'))
def _(utterance):
    return TextAction('○ 주택용누진제는 "73년 석유파동을 계기로 에너지 다소비층에 대한 소비절약 유도와 사회적 취약계층인 저소득층 보호를 위하여 시행되었습니다.○ 이와 달리, 주택용 이외의 고객은 업무효율화와 전기요금 변화에 따른 사업리스크를 감소시키기 위하여 누진제를 시행하고 있지 않습니다. 하지만, 전기요금의 기본요금을 주택용과 비교하여 상대적으로 높게 책정하고, 「초과사용부가금」제도와 「계절ㆍ시간대별 차등요금제」등으로 소비절약과 합리적인 전기사용을 유도하고 있습니다.')


@agent.on(Domain('전기/요금'))
@agent.on(Intent('문의하다/HOW > 누진제'))
def _(utterance):
    return TextAction('주택용전력의 전기요금은 다소비 가구의 소비절약을 유도하기 위해 많이 쓰면 쓸수록 단가가 비싸지는 누진요금제도로 되어 있습니다. 누진율은 사용량에 따라 기본요금은 3단계로, 전력량요금은 3단계로 적용합니다. ▣ 주택용전력(저압) 주택용전력(저압) 기본요금(원/호) 전력량요금(원/㎾h) 200kWh 이하 사용 910 처음 200kWh까지 93.3 201 ∼ 400kWh 사용 1,600 다음 200kWh까지 187.9 400kWh 초과 사용 7,300 400kWh 초과 280.6 * 필수사용량 보장공제 : 200kWh 이하 사용시 월 4,000원 한도 감액 (감액 후 최저요금 1,000원)* 슈퍼유저요금 : 동·하계(7~8월, 12~2월) 1,000kWh 초과 전력량요금은 709.5원/kWh 적용 - 월간 최저요금은 1,000원으로 합니다.▣ 주택용전력(고압) 주택용전력(고압) 기본요금(원/호) 전력량요금(원/㎾h) 200kWh 이하 사용 730 처음 200kWh까지 78.3 201 ∼ 400kWh 사용 1,260 다음 200kWh까지 147.3 400kWh 초과 사용 6,060 400kWh 초과 215.6 * 필수사용량 보장공제 : 200kWh 이하 사용시 월 2,500원 한도 감액 (감액 후 최저요금 1,000원)* 슈퍼유저요금 : 동·하계(7~8월, 12~2월) 1,000kWh 초과 전력량요금은 574.6원/kWh 적용')

In [23]:
from kent.utterance import Utterance

while True:
    utterance = Utterance(input('In: '))
    responses = agent.listen(utterance)
    
    print(f'- {utterance.domain}')
    print(f'- {utterance.intent}')
    
    print('Out:')
    for response in responses:
        for action in response.actions:
            print(action.text)
    print()

In:  왜 주택용 전기 요금만 누진제를 적용하나요?


- Domain(name='전기/요금', confidence=0.9990919375754238)
- Intent(name='문의하다/WHY > 누진제', confidence=0.9990898817036976)
Out:
○ 주택용누진제는 "73년 석유파동을 계기로 에너지 다소비층에 대한 소비절약 유도와 사회적 취약계층인 저소득층 보호를 위하여 시행되었습니다.○ 이와 달리, 주택용 이외의 고객은 업무효율화와 전기요금 변화에 따른 사업리스크를 감소시키기 위하여 누진제를 시행하고 있지 않습니다. 하지만, 전기요금의 기본요금을 주택용과 비교하여 상대적으로 높게 책정하고, 「초과사용부가금」제도와 「계절ㆍ시간대별 차등요금제」등으로 소비절약과 합리적인 전기사용을 유도하고 있습니다.



In:  누진 요금은 어떻게 계산되나요?


- Domain(name='전기/요금', confidence=0.9961842853624098)
- Intent(name='문의하다/HOW > 계산/전기료', confidence=0.8693960991719217)
Out:
전기요금은 기본요금과 전력량요금으로 구성되며, 기본요금과 전력량요금의 합계에 전력산업기반기금과 부가가치세가 포함되어 청구금액이 결정됩니다. 기본요금 및 전력량요금 단가는 전기공급방식(고압, 저압), 계약종별(주택용, 일반용, 산업용, 교육용, 농사용 등)에 따라 다릅니다. 주택용전력은 사용량에 따라 기본요금은 3단계, 전력량요금은 3단계로 구분하여 누진율을 적용합니다. 주택용전력을 제외한 모든 계약종별의 기본요금 적용은 계약전력을 기준으로 하므로 계약전력은 요금계산의 기준이 되는 요금적용전력이 됩니다. 다만, 최대수요전력계를 설치한 고객에 대하여는 검침 당월을 포함한 직전 12개월중 12월분, 1월분, 2월분, 7월분, 8월분, 9월분 및 당월분의 최대수요전력 중 가장 큰 최대수요전력을 요금적용전력으로 하여 기본요금을 산정합니다. "사이버지점>전기요금계산기"을 클릭하시면 사용량에 따른 전기요금을 계산해 보실 수 있습니다. 또 "사이버지점>조회·납부>전기요금표"에서는 계약종별별 전기요금표를 보실 수 있습니다.◆ 주택용전력 저압(주거용) 전기요금 계산 예 : 225kW 사용시① 기본요금 : 200kWh초과로 1,600원 ② 전력량요금 : 23,357원 - 처음 200kWh×93.3원=18,660원 - 다음 25kWh×187.9원=4,697.5원 (원미만 절사)③ 요금합계 (기본요금+전력량요금) : 1,600원+23,357원=24,957원 ④ 부가가치세 (요금합계*10%) : 24,957원×0.1=2,496원 (원미만 4사5입)⑤ 전력산업기반기금 (요금합계×3.7%) : 24,957원×0.037=920원 (10원미만절사)⑥ 청구금액：24,957원+2,496원+920원=28,370원 (국고금단수법에 의해 10원미만 절사)



In:  누진 요금 계산 방법 알려주세요


- Domain(name='전기/요금', confidence=0.9961642013270161)
- Intent(name='문의하다/HOW > 계산/전기료', confidence=0.9358964536260758)
Out:
전기요금은 기본요금과 전력량요금으로 구성되며, 기본요금과 전력량요금의 합계에 전력산업기반기금과 부가가치세가 포함되어 청구금액이 결정됩니다. 기본요금 및 전력량요금 단가는 전기공급방식(고압, 저압), 계약종별(주택용, 일반용, 산업용, 교육용, 농사용 등)에 따라 다릅니다. 주택용전력은 사용량에 따라 기본요금은 3단계, 전력량요금은 3단계로 구분하여 누진율을 적용합니다. 주택용전력을 제외한 모든 계약종별의 기본요금 적용은 계약전력을 기준으로 하므로 계약전력은 요금계산의 기준이 되는 요금적용전력이 됩니다. 다만, 최대수요전력계를 설치한 고객에 대하여는 검침 당월을 포함한 직전 12개월중 12월분, 1월분, 2월분, 7월분, 8월분, 9월분 및 당월분의 최대수요전력 중 가장 큰 최대수요전력을 요금적용전력으로 하여 기본요금을 산정합니다. "사이버지점>전기요금계산기"을 클릭하시면 사용량에 따른 전기요금을 계산해 보실 수 있습니다. 또 "사이버지점>조회·납부>전기요금표"에서는 계약종별별 전기요금표를 보실 수 있습니다.◆ 주택용전력 저압(주거용) 전기요금 계산 예 : 225kW 사용시① 기본요금 : 200kWh초과로 1,600원 ② 전력량요금 : 23,357원 - 처음 200kWh×93.3원=18,660원 - 다음 25kWh×187.9원=4,697.5원 (원미만 절사)③ 요금합계 (기본요금+전력량요금) : 1,600원+23,357원=24,957원 ④ 부가가치세 (요금합계*10%) : 24,957원×0.1=2,496원 (원미만 4사5입)⑤ 전력산업기반기금 (요금합계×3.7%) : 24,957원×0.037=920원 (10원미만절사)⑥ 청구금액：24,957원+2,496원+920원=28,370원 (국고금단수법에 의해 10원미만 절사)



In:  누진 금액은 어떻게 계산되나요?


- Domain(name='전기/요금', confidence=0.9980345515806529)
- Intent(name='문의하다/HOW > 누진제', confidence=0.9599205774063735)
Out:
주택용전력의 전기요금은 다소비 가구의 소비절약을 유도하기 위해 많이 쓰면 쓸수록 단가가 비싸지는 누진요금제도로 되어 있습니다. 누진율은 사용량에 따라 기본요금은 3단계로, 전력량요금은 3단계로 적용합니다. ▣ 주택용전력(저압) 주택용전력(저압) 기본요금(원/호) 전력량요금(원/㎾h) 200kWh 이하 사용 910 처음 200kWh까지 93.3 201 ∼ 400kWh 사용 1,600 다음 200kWh까지 187.9 400kWh 초과 사용 7,300 400kWh 초과 280.6 * 필수사용량 보장공제 : 200kWh 이하 사용시 월 4,000원 한도 감액 (감액 후 최저요금 1,000원)* 슈퍼유저요금 : 동·하계(7~8월, 12~2월) 1,000kWh 초과 전력량요금은 709.5원/kWh 적용 - 월간 최저요금은 1,000원으로 합니다.▣ 주택용전력(고압) 주택용전력(고압) 기본요금(원/호) 전력량요금(원/㎾h) 200kWh 이하 사용 730 처음 200kWh까지 78.3 201 ∼ 400kWh 사용 1,260 다음 200kWh까지 147.3 400kWh 초과 사용 6,060 400kWh 초과 215.6 * 필수사용량 보장공제 : 200kWh 이하 사용시 월 2,500원 한도 감액 (감액 후 최저요금 1,000원)* 슈퍼유저요금 : 동·하계(7~8월, 12~2월) 1,000kWh 초과 전력량요금은 574.6원/kWh 적용



KeyboardInterrupt: 